<a href="https://colab.research.google.com/github/wanburana/AT82.01-brain-project-group-4/blob/main/Jirasak/code_not_used/cwt_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne
!gdown --id 1LoyiM9bQlrullvtoPA7X4FWhNDBGPnED
!unzip /content/data_preprocessed_python.zip -d data_preprocessed_python

     |████████████████████████████████| 7.4 MB 6.5 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1LoyiM9bQlrullvtoPA7X4FWhNDBGPnED
To: /content/data_preprocessed_python.zip
100% 3.30G/3.30G [00:49<00:00, 66.3MB/s]
Archive:  /content/data_preprocessed_python.zip
 extracting: data_preprocessed_python/s01.dat  
 extracting: data_preprocessed_python/s02.dat  
 extracting: data_preprocessed_python/s03.dat  
 extracting: data_preprocessed_python/s04.dat  
 extracting: data_preprocessed_python/s05.dat  
 extracting: data_preprocessed_python/s06.dat  
 extracting: data_preprocessed_python/s07.dat  
 extracting: data_preprocessed_python/s08.dat  
 extracting: data_preprocessed_python/s09.dat  
 extracting: data_preprocessed_python/s10.dat  
 extracting: data_preprocessed_python/s11.dat  
 extracting: data_preprocessed_python/s12.dat  
 extracting: data_preprocessed_python/s13.dat  
 extracting: data_preprocessed_python/s14.dat  
 extracting: data_preprocessed_python/s15.dat  
 extra

In [ ]:
import os
from glob import glob
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import mne
import pywt

In [ ]:
FOLDER_PATH = '/content/data_preprocessed_python/'
files_list = sorted(glob(FOLDER_PATH + '*'))

In [ ]:
features = []
targets = []

for i, f in (enumerate(tqdm(files_list))):
    with open(f, 'rb') as f: 
        content = pickle.load(f, encoding='latin1')
        data = content['data']
        labels = content['labels']
        
        n_clips, n_channels, len_features = data.shape
        
        for j in range(n_clips):
            features.append(data[j])
            targets.append(labels[j])

features = np.array(features) 
targets = np.array(targets)     

eeg_features = features[:, :32, :] # use only eeg channels
print(eeg_features.shape)

100%|██████████| 32/32 [00:05<00:00,  6.19it/s]


(1280, 32, 8064)


In [ ]:
freq = 128
time = 63 # 60s + 3s
baseline_time = 3 # 3 second at the start of the trial

train_size = eeg_features.shape[0]
n_channels = eeg_features.shape[1]
seq_len = eeg_features.shape[2] - baseline_time * freq # remove baseline from the clip



In [ ]:
# STYLE 2
# ONLY FIRST 60 FRAME?

# CWT
scales = range(1,65)
waveletname = 'morl'
train_size = eeg_features.shape[0]
n_channels = eeg_features.shape[1]
seq_len = eeg_features.shape[2]
train_data_cwt = np.ndarray(shape=(train_size, n_channels, len(scales), seq_len))


Fs = 128;
Time = 63;
frameNum = 60;
totalScale = 64;
exScale = 32;
wname = 'db4';

final_output = np.zeros((train_size, 1024, 60));  
datastart = 128*3;
datalength = 8064 - datastart;

for ii in tqdm(range(0,train_size)):

    output = np.zeros((n_channels, totalScale, frameNum));  
    for jj in range(n_channels):
        signal = eeg_features[ii, jj, : ]
        coeff, freq = pywt.cwt(signal[datastart:], np.arange(1,totalScale+1), waveletname);

        frameSize = 60//frameNum;
        start = 0
        for kk in range(frameNum):
            # output[jj,:,kk] = np.sum(coeff[:,kk*Fs:kk*Fs + Fs], 1)
            output[jj,:,kk] = np.sum(coeff[:,kk:frameSize*kk+1], 1);
    #         start = start + frameSize;
    #     # output[jj, :, :] = coeff_compressed
    exoutput = output[:,8:40,:] # select only scales of 8 - 39
    output2d = exoutput.reshape(-1, 60); # 1024, 60
    final_output[ii,:,:] = output2d
    
    # if ii >= 10:
    #     break
np.save('/content/drive/MyDrive/AT82.01 Project/style2.npy', final_output)

100%|██████████| 1280/1280 [39:12<00:00,  1.84s/it]
